In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_excel('./data.xlsx')
data.head()

,iID,iPID,uFILE_FLAG,sARCH_ID,sFLOW_NAME,sNODE_NAME,iNODE_STATUS,iUSER_ID,iWF_ID,iWN_ID,...,iFLOW_NODE_NO,iPROC_USERID,sPIC_SERVER_PATH,sPDF_SERVER_PATH,iARCH_TYPE,sORDER_ARCH_ID,dUPDATE_TIME,dNODE_TIME,dPROC_TIME,sBatch_number
0,725975,186250,0B71F6DD-E7EC-4FB2-87C4-EB576B70F4C2,托644031-册一,不动产工作流,扫描,2,68,20,12,...,1,NaN,\\127.0.0.1\tlscan\tlscanfile,\\192.168.1.253\tlscan\tlscanpdf,91,托00644031-册001,2020-07-01 09:17:22.313,2020-07-01 09:21:10,NaT,902
1,725976,186251,7B0B1758-12F7-4A44-91F9-96C5B9B2E6FD,托644032-册一,不动产工作流,扫描,2,68,20,12,...,1,NaN,\\127.0.0.1\tlscan\tlscanfile,\\192.168.1.253\tlscan\tlscanpdf,91,托00644032-册001,2020-07-01 09:17:42.867,2020-07-01 09:21:17,NaT,902
2,725249,185781,94CC72C0-A6B4-4627-868E-EE3B423877C7,托7181-册一,不动产工作流,扫描,2,44,20,12,...,1,NaN,\\127.0.0.1\tlscan\tlscanfile,\\192.168.1.253\tlscan\tlscanpdf,91,托00007181-册001,2020-07-01 08:39:08.370,2020-07-01 09:26:29,NaT,604
3,725250,185782,FA6E9CC1-5579-41D5-918B-6B9F0D3B2486,托7182-册一,不动产工作流,扫描,2,44,20,12,...,1,NaN,\\127.0.0.1\tlscan\tlscanfile,\\192.168.1.253\tlscan\tlscanpdf,91,托00007182-册001,2020-07-01 08:39:08.397,2020-07-01 09:26:36,NaT,604
4,725251,185783,01016875-EA2C-43BE-9637-CD649F21E58A,托7183-册一,不动产工作流,扫描,2,44,20,12,...,1,NaN,\\127.0.0.1\tlscan\tlscanfile,\\192.168.1.253\tlscan\tlscanpdf,91,托00007183-册001,2020-07-01 08:39:08.420,2020-07-01 09:26:42,NaT,604


In [28]:
df_first.dropna().info()  # 时间里面存在缺失值

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33980 entries, 0 to 33984
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   案件号        33980 non-null  object        
 1   扫描开始时间     33980 non-null  datetime64[ns]
 2   扫描结束时间     33980 non-null  datetime64[ns]
 3   图像处理开始时间   33980 non-null  datetime64[ns]
 4   图像处理结束时间   33980 non-null  datetime64[ns]
 5   自检全检开始时间   33980 non-null  datetime64[ns]
 6   自检全检结束时间   33980 non-null  datetime64[ns]
 7   PDF处理开始时间  33980 non-null  datetime64[ns]
 8   PDF处理结束时间  33980 non-null  datetime64[ns]
 9   完成时长       33980 non-null  float64       
dtypes: datetime64[ns](8), float64(1), object(1)
memory usage: 2.9+ MB


In [29]:
## 时间归类
scan = data[data['sNODE_NAME'].eq('扫描')][['sARCH_ID', 'dUPDATE_TIME', 'dNODE_TIME']].rename(columns={'dUPDATE_TIME': '扫描开始时间', 'dNODE_TIME': '扫描结束时间'})

img_pro = data[data['sNODE_NAME'].eq('图像处理')][['sARCH_ID', 'dUPDATE_TIME', 'dNODE_TIME']].rename(columns={'dUPDATE_TIME': '图像处理开始时间', 'dNODE_TIME': '图像处理结束时间'})

self_checked = data[data['sNODE_NAME'].eq('自检全检')][['sARCH_ID', 'dUPDATE_TIME', 'dNODE_TIME']].rename(columns={'dUPDATE_TIME': '自检全检开始时间', 'dNODE_TIME': '自检全检结束时间'})

pdf_pro = data[data['sNODE_NAME'].eq('PDF处理')][['sARCH_ID', 'dUPDATE_TIME', 'dNODE_TIME']].rename(columns={'dUPDATE_TIME': 'PDF处理开始时间', 'dNODE_TIME': 'PDF处理结束时间'})

In [30]:
## 数据合并
df_first = pd.merge(scan, img_pro, on='sARCH_ID', how='outer')
df_first = pd.merge(df_first, self_checked, on='sARCH_ID', how='outer')
df_first = pd.merge(df_first, pdf_pro, on='sARCH_ID', how='outer')

In [31]:
df_first = df_first.rename(columns={'sARCH_ID': '案件号'})
df_first.head()

,案件号,扫描开始时间,扫描结束时间,图像处理开始时间,图像处理结束时间,自检全检开始时间,自检全检结束时间,PDF处理开始时间,PDF处理结束时间
0,托644031-册一,2020-07-01 09:17:22.313,2020-07-01 09:21:10,2020-07-01 15:01:56,2020-07-01 16:44:24,2020-07-08 08:35:26,2020-07-08 10:54:15,2020-07-10 15:05:18,2020-07-10 15:18:02
1,托644032-册一,2020-07-01 09:17:42.867,2020-07-01 09:21:17,2020-07-01 15:01:56,2020-07-01 16:44:26,2020-07-08 08:35:26,2020-07-08 10:54:17,2020-07-10 15:05:18,2020-07-10 15:18:02
2,托7181-册一,2020-07-01 08:39:08.370,2020-07-01 09:26:29,2020-07-02 10:47:50,2020-07-02 11:30:47,2020-07-02 11:31:18,2020-07-02 11:43:48,2020-07-03 08:49:13,2020-07-03 09:04:24
3,托7182-册一,2020-07-01 08:39:08.397,2020-07-01 09:26:36,2020-07-02 10:47:50,2020-07-02 11:31:08,2020-07-02 11:31:18,2020-07-02 11:43:50,2020-07-03 08:49:13,2020-07-03 09:04:24
4,托7183-册一,2020-07-01 08:39:08.420,2020-07-01 09:26:42,2020-07-02 13:37:15,2020-07-02 13:53:05,2020-07-02 15:07:37,2020-07-02 15:15:12,2020-07-03 08:49:13,2020-07-03 09:04:24


In [32]:
df_first['完成时长'] = 0

In [52]:
for i in range(0,df_first.shape[0]):
    scan_diff = df_first.loc[i,'扫描结束时间'] - df_first.loc[i,'扫描开始时间']
    img_diff = df_first.loc[i,'图像处理结束时间'] - df_first.loc[i,'图像处理开始时间']
    assess_diff = df_first.loc[i,'自检全检结束时间'] - df_first.loc[i,'自检全检开始时间']
    df_first.loc[i,'完成时长'] = round(((scan_diff+img_diff+assess_diff).total_seconds()/3600),3)

In [34]:
df_first.head()

,案件号,扫描开始时间,扫描结束时间,图像处理开始时间,图像处理结束时间,自检全检开始时间,自检全检结束时间,PDF处理开始时间,PDF处理结束时间,完成时长
0,托644031-册一,2020-07-01 09:17:22.313,2020-07-01 09:21:10,2020-07-01 15:01:56,2020-07-01 16:44:24,2020-07-08 08:35:26,2020-07-08 10:54:15,2020-07-10 15:05:18,2020-07-10 15:18:02,4.085
1,托644032-册一,2020-07-01 09:17:42.867,2020-07-01 09:21:17,2020-07-01 15:01:56,2020-07-01 16:44:26,2020-07-08 08:35:26,2020-07-08 10:54:17,2020-07-10 15:05:18,2020-07-10 15:18:02,4.082
2,托7181-册一,2020-07-01 08:39:08.370,2020-07-01 09:26:29,2020-07-02 10:47:50,2020-07-02 11:30:47,2020-07-02 11:31:18,2020-07-02 11:43:48,2020-07-03 08:49:13,2020-07-03 09:04:24,1.713
3,托7182-册一,2020-07-01 08:39:08.397,2020-07-01 09:26:36,2020-07-02 10:47:50,2020-07-02 11:31:08,2020-07-02 11:31:18,2020-07-02 11:43:50,2020-07-03 08:49:13,2020-07-03 09:04:24,1.722
4,托7183-册一,2020-07-01 08:39:08.420,2020-07-01 09:26:42,2020-07-02 13:37:15,2020-07-02 13:53:05,2020-07-02 15:07:37,2020-07-02 15:15:12,2020-07-03 08:49:13,2020-07-03 09:04:24,1.183


In [48]:
df_first.to_excel('./result1_1.xlsx',index=False)

## 任务1.2

In [287]:
ret = data[(data['iNODE_STATUS'] == 5)].shape[0]
fin = data[(data['iNODE_STATUS'] == 2) | (data['iNODE_STATUS'] == 5)].shape[0]
proporation = (ret/fin) *100
print('需要返工的案卷数量为:'+str(ret))
print('需要返工的案卷数量及其占完工案卷总数的百分比为：'+str(proporation))

需要返工的案卷数量为:797
需要返工的案卷数量及其占完工案卷总数的百分比为：0.5862880682654112


In [40]:
scan_2 = data[data['sNODE_NAME'].eq('扫描')][['sARCH_ID', 'dUPDATE_TIME']].rename(columns={'dUPDATE_TIME': '扫描' })

img_pro_2 = data[data['sNODE_NAME'].eq('图像处理')][['sARCH_ID', 'dUPDATE_TIME']].rename(columns={'dUPDATE_TIME': '图像处理'})

self_checked_2 = data[data['sNODE_NAME'].eq('自检全检')][['sARCH_ID', 'dUPDATE_TIME']].rename(columns={'dUPDATE_TIME': '自检全检'})

pdf_pro_2 = data[data['sNODE_NAME'].eq('PDF处理')][['sARCH_ID', 'dUPDATE_TIME']].rename(columns={'dUPDATE_TIME': 'PDF处理'})

In [41]:
df_second = pd.merge(scan_2, img_pro_2, on='sARCH_ID', how='outer')
df_second = pd.merge(df_second, self_checked_2, on='sARCH_ID', how='outer')
df_second = pd.merge(df_second, pdf_pro_2, on='sARCH_ID', how='outer')
df_second = df_second.rename(columns={'sARCH_ID': '案件号'})
df_second.head()

,案件号,扫描,图像处理,自检全检,PDF处理
0,托644031-册一,2020-07-01 09:17:22.313,2020-07-01 15:01:56,2020-07-08 08:35:26,2020-07-10 15:05:18
1,托644032-册一,2020-07-01 09:17:42.867,2020-07-01 15:01:56,2020-07-08 08:35:26,2020-07-10 15:05:18
2,托7181-册一,2020-07-01 08:39:08.370,2020-07-02 10:47:50,2020-07-02 11:31:18,2020-07-03 08:49:13
3,托7182-册一,2020-07-01 08:39:08.397,2020-07-02 10:47:50,2020-07-02 11:31:18,2020-07-03 08:49:13
4,托7183-册一,2020-07-01 08:39:08.420,2020-07-02 13:37:15,2020-07-02 15:07:37,2020-07-03 08:49:13


In [49]:
df_second.to_excel('./result1_2.xlsx',index=False)

## 任务1.3

In [312]:
worker_ret = data[data['iNODE_STATUS']==5]
worker_ret_ass = worker_ret[worker_ret['iFLOW_NODE_NO']==3]
worker_ret_ass = worker_ret_ass.groupby(['iPROC_USERID'])['num'].sum()
worker_ret_ass =worker_ret_ass.reset_index(drop = False)
ass_list = list(worker_ret_ass['iPROC_USERID'].values)

In [313]:
data['num'] = 1
worker_ret = data[data['iNODE_STATUS']==5]
df_third = worker_ret
worker_ret = data.groupby(['iPROC_USERID'])['num'].sum()
worker_ret = worker_ret.reset_index(drop = False)

In [314]:
df_third = pd.merge(worker_ret, worker_ret_ass, on='iPROC_USERID', how='outer')
df_third = df_third.dropna()

In [315]:
df_third['diff'] = df_third['num_x']-df_third['num_y']
df_third['返工案卷占比 (%)'] = round(df_third['diff']/df_third['num_x'],3)
df_third

,iPROC_USERID,num_x,num_y,diff,返工案卷占比 (%)
0,10.0,109,85.0,24.0,0.220
2,12.0,107,107.0,0.0,0.000
3,13.0,39,36.0,3.0,0.077
4,17.0,1,1.0,0.0,0.000
9,42.0,347,135.0,212.0,0.611
21,87.0,2,2.0,0.0,0.000


In [316]:
df_third_drop=df_third.drop(['num_x','num_y','diff'],axis=1)
df_third_drop = df_third_drop.rename(columns={'iPROC_USERID':'操作人员 ID'})

In [317]:
df_third_drop.reset_index(drop = True) 

,操作人员 ID,返工案卷占比 (%)
0,10.0,0.220
1,12.0,0.000
2,13.0,0.077
3,17.0,0.000
4,42.0,0.611
5,87.0,0.000


In [318]:
df_third_drop.to_excel('./result1_3.xlsx',index=False)

## 任务1.4

In [410]:
def is_workday(date):
    weekdays = [0, 1, 2, 3, 4,5]  # 周一到周六
    return date.weekday() in weekdays

In [411]:
from datetime import datetime, timedelta
# 转换时间为datetime类型  
data['dUPDATE_TIME'] = pd.to_datetime(data['dUPDATE_TIME'])
data['dNODE_TIME'] = pd.to_datetime(data['dNODE_TIME'])
data['dPROC_TIME'] = pd.to_datetime(data['dPROC_TIME'])

In [412]:
# 已完成并提交的案卷  
ret = data[(data['iNODE_STATUS'] == 2) | (data['iNODE_STATUS'] == 5)]

# 计算每批的总耗时  
data['total_consume'] = data['dNODE_TIME'] - data['dUPDATE_TIME']

# 计算每批中每个案卷的工作日时间  
data['workday_time'] = data.apply(lambda row: row['total_consume'] if is_workday(row['dUPDATE_TIME'].date()) 
                                  and is_workday(row['dNODE_TIME'].date()) else timedelta(0), axis=1)

# 将工作日时间转换为小时  
data['workday_time_hours'] = data['workday_time'].dt.total_seconds() / 3600

In [413]:
# 根据工序分组计算完成案卷的数量、工作日总耗时和平均耗时  
df_forth = data.groupby('iFLOW_NODE_NO').agg({'iID': 'count', 'workday_time_hours': 'sum', 'sBatch_number': 'first'}).reset_index()

# 计算平均耗时  
df_forth['avg_time_hours'] = df_forth['workday_time_hours'] / df_forth['iID']
df_forth = df_forth.drop('sBatch_number',axis=1)

# 重命名列名  
df_forth.columns = ['工序', '完成案卷的数量', '总耗时 (h)', '平均耗时 (h/卷)']

# 结果保留3位小数  
df_forth = df_forth.round(3)

# map
map = {1:'扫描',2:'图像处理',3:'自检全检',4:'PDF处理'}

df_forth['工序'] = df_forth['工序'].map(map)

In [414]:
df_forth

,工序,完成案卷的数量,总耗时 (h),平均耗时 (h/卷)
0,扫描,33985,475045.736,13.978
1,图像处理,33985,240986.654,7.091
2,自检全检,33985,183827.083,5.409
3,PDF处理,33985,118067.318,3.474


In [415]:
df_forth.to_excel('./result1_4.xlsx',index=False)

## 任务1.5

In [416]:
data.loc[data['dUPDATE_TIME'].dt.hour < 8.5,'dUPDATE_TIME'] = data['dUPDATE_TIME'].dt.date + pd.Timedelta(hours=8)

data.loc[(data['dUPDATE_TIME'].dt.hour >= 12) & 
         (data['dUPDATE_TIME'].dt.hour < 13),'dUPDATE_TIME'] = data['dUPDATE_TIME'].dt.date + pd.Timedelta(hours=13)

data.loc[(data['dNODE_TIME'].dt.hour > 12) &
         (data['dNODE_TIME'].dt.hour < 13),'dNODE_TIME'] = data['dNODE_TIME'].dt.date + pd.Timedelta(hours=12)

data.loc[data['dNODE_TIME'].dt.hour > 18,'dNODE_TIME'] = data['dNODE_TIME'].dt.date + pd.Timedelta(hours=18)

In [417]:
df_fifth = data.groupby(['iUSER_ID', 'iFLOW_NODE_NO', 'sBatch_number'])
df_fifth = df_fifth.agg({'iUSER_ID': 'count' ,'dUPDATE_TIME':'first', 'dNODE_TIME':'last'})
df_fifth.columns = ['案卷完成数', '开始时间', '结束时间']

df_fifth['案卷耗时(h)'] = (df_fifth['结束时间'] - df_fifth['开始时间']).dt.total_seconds()/3600
df_fifth['每个案卷的平均耗时 (h/卷)'] =  df_fifth['案卷完成数']/df_fifth['案卷耗时(h)']
df_fifth.reset_index(inplace=True)
df_fifth

,iUSER_ID,iFLOW_NODE_NO,sBatch_number,案卷完成数,开始时间,结束时间,案卷耗时(h),每个案卷的平均耗时 (h/卷)
0,8,2,12,24,2020-07-03 14:18:38,2020-07-03 14:36:54,0.304444,78.832117
1,8,3,13,13,2020-07-17 00:00:00,2020-07-17 12:32:50,12.547222,1.036086
2,8,4,0,1,2020-07-06 16:51:43,2020-07-06 16:52:13,0.008333,120.000000
3,8,4,1,668,2020-07-14 17:12:32,2020-07-17 12:35:41,67.385833,9.913063
4,8,4,2,2,2020-07-23 15:20:06,2020-07-23 17:27:59,2.131389,0.938355
...,...,...,...,...,...,...,...,...
1628,98,2,1628,25,2020-07-29 14:59:34,2020-07-29 15:28:01,0.474167,52.724077
1629,98,2,1629,25,2020-07-29 16:53:31,2020-07-30 09:02:05,16.142778,1.548680
1630,98,2,1630,25,2020-07-30 09:03:30,2020-07-30 09:28:37,0.418611,59.721301
1631,98,2,1631,25,2020-07-30 10:50:18,2020-07-30 11:36:36,0.771667,32.397408


In [418]:
df_fifth.sort_values('iUSER_ID', inplace=True)
df_fifth = df_fifth.reset_index(drop=True)
df_fifth = df_fifth.drop(['sBatch_number', '开始时间', '结束时间'],axis =1)

# 重命名列名  
df_fifth.rename(columns={'iUSER_ID': '操作人员ID', 'iFLOW_NODE_NO': '工序', '案卷耗时(h)': '工作时长(h)', '案卷完成数': '完成案卷的数量'}, inplace=True)

#map
map = {1:'扫描',2:'图像处理',3:'自检全检',4:'PDF处理'}

df_fifth['工序'] = df_fifth['工序'].map(map)

In [419]:
df_fifth.to_excel('./result1_5.xlsx',index=False)

In [420]:
select_5 = df_fifth[(df_fifth['操作人员ID'] == 10)|(df_fifth['操作人员ID'] == 33)|(df_fifth['操作人员ID'] == 48)]
select_5.round(3)

,操作人员ID,工序,完成案卷的数量,工作时长(h),每个案卷的平均耗时 (h/卷)
14,10,自检全检,46,0.465,98.925
15,10,自检全检,100,1.205,82.968
16,10,自检全检,19,0.379,50.110
17,10,自检全检,81,0.982,82.489
18,10,自检全检,100,2.033,49.180
...,...,...,...,...,...
801,48,图像处理,75,4.360,17.201
802,48,图像处理,24,1.113,21.568
803,48,图像处理,100,42.497,2.353
804,48,图像处理,44,2.142,20.537
